<h1 align="center"> 2021 Tokyo Olimpic Medals EDA </h1>

<img src="https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcT2J5-y2wFs9m8RNaAyNRLCOsddU0rsPe7V7w&usqp=CAU" width="50%" style="text-align: center;"/>

Created: 2021-08-12 || Last updated: 2021-08-12

Kaggle Kernel made by 🚀 <a href="https://www.kaggle.com/rafanthx13"> Rafael Morais de Assis</a>

Dataset of Population from from [worlOmeter](https://www.worldometers.info/coronavirus/#countries)

## Table Of Content (TOC) <a id="top"></a>

+ [Import Libs](#index01) 
+ [Snippets](#index02)
+ [Import DataSet](#index03)
+ [Import GeoJson Of World](#index04)
+ [EDA: Map of Count Medals on World](#index05)
+ [Web Scraping of Country Population](#index06)
+ [EDA: Medals by Population](#index07)

  
<!-- COMMENTS: OBS: Criado em 11/08/2021 => Ver a proporçao de medalhas de ouro por habitante, e total de medalahes; Usar populaçao do 'COVID19 WorldOMeter TEMP'; Renovar tambem o kaggle 'COVID19 WorldOMeter TEMP'; LEMBRAR DE COMENTAR MIA,S PORQUE ALUNGS SNIPPETS JÁ ESTAO FICANDO CONFSUSO DE LER, DEPOIS DE QUASE 1 ANO FEITO. -->

## Import Libs <a id='index01'></a> <a href="#top" style="background-color: #20beff; border: none; color: white;  padding: 15px 30px;  font-family: sans-serif;  border-radius: 12px;  text-align: center;  text-decoration: none;  display: inline-block;  font-size: 16px;  margin: 4px 2px;  cursor: pointer;" >Go to TOC</a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random

import warnings
warnings.filterwarnings("ignore")

# Default Configs
pd.options.display.float_format = '{:,.4f}'.format # format float
sns.set(style="whitegrid") ## white background on seaborn
plt.style.use('seaborn')
# Random Number
seed = 42
np.random.seed(seed)
random.seed(seed)

## Snippets <a id='index02'></a> <a href="#top" style="background-color: #20beff; border: none; color: white;  padding: 15px 30px;  font-family: sans-serif;  border-radius: 12px;  text-align: center;  text-decoration: none;  display: inline-block;  font-size: 16px;  margin: 4px 2px;  cursor: pointer;" >Go to TOC</a>

In [ ]:
from bokeh.io import show
from bokeh.plotting import figure
from bokeh.models import LinearColorMapper, HoverTool, ColorBar
from bokeh.palettes import magma,viridis,cividis, inferno
from bokeh.models import WheelZoomTool, BoxZoomTool, ResetTool

def eda_us_states_geo_plot(geosource, df_in, title, column, state_column, low = -1, high = -1, palette = -1, plot_width=500):
    """
    Generate Bokeh Plot to Brazil States:
        geosource: GeoJSONDataSource of Bokeh
        df_in: DataSet before transformed in GeoJSONDataSource
        title: title of plot
        column: column of df_in to be placed values in geoplot
        state_column: indicate column with names of States
        low = (optional) min value of range of color spectre
        high = (optional) max values of range of color spectre
        palette: (optional) can be magma, viridis, civis, inferno e etc.. (with number os colors)
            Example: cividis(8) (8 colors to classify), cividis(256)  (256, more colors to clasify)
    """
    if high == -1:
        print(df_in.columns.tolist())
        df_in[column] = pd.to_numeric(df_in[column],errors = 'coerce')
        high = max(df_in[column])
    if low == -1:
        low = min(df_in[column])
    if palette == -1:
        palette = inferno(24)
        
    palette = palette[::-1]
    color_mapper = LinearColorMapper(palette = palette, low = low, high = high)
    
    hover = HoverTool(tooltips = [ ('Team/NOC','@{'+'name'+'}'), (column, '@{'+column+'}{%.2f}')],
                  formatters={'@{'+column+'}' : 'printf'})

    color_bar = ColorBar(color_mapper=color_mapper, label_standoff=8, width = 450, height = 20, 
                         border_line_color=None, location = (0,0),  orientation = 'horizontal')

    p = figure(title = title, plot_height = 400, plot_width = plot_width, tools = [hover])

    p.xgrid.grid_line_color = None
    p.ygrid.grid_line_color = None
    p.xaxis.visible = False
    p.yaxis.visible = False

    p.patches('xs','ys', source = geosource, line_color = 'black', line_width = 0.25,
              fill_alpha = 1, fill_color = {'field' : str(column), 'transform' : color_mapper})

    p.add_layout(color_bar, 'below')
    p.add_tools(WheelZoomTool())
    p.add_tools(ResetTool())
    return p   

from bokeh.palettes import Turbo256 
from bokeh.models import ColumnDataSource
from bokeh.transform import factor_cmap
from bokeh.palettes import magma,viridis,cividis, inferno

def eda_bokeh_horiz_bar_ranked(df, column_target, title = '', int_top = 3, second_target = 'Team/NOC'):
    """
    Generate Bokeh Plot ranking top fists and last value:
        df: data_frame
        column_targe: a column of df inputed
        title: title of plot
        int_top: number of the tops
        column: column of df_in to be placed values in geoplot
        second_targe = 'state'
    """
    ranked = df.sort_values(by=column_target, ascending=False).reset_index(drop = True)
    ranked = ranked.dropna()
    top_int = int_top
    # top = ranked[:top_int].append(ranked[-top_int:]) # Bottom an Top
    top = ranked[:top_int+top_int]# only Top
    top.index = top.index + 1
    source = ColumnDataSource(data=top)
    list_second_target = source.data[second_target].tolist()
    index_label = list_second_target[::-1] # reverse order label

    p = figure(plot_width=500, plot_height=400, y_range=index_label, 
                toolbar_location=None, title=title)   

    p.hbar(y=second_target, right=column_target, source=source, height=0.85, line_color="#000000",
          fill_color=factor_cmap(second_target, palette=inferno(24), factors=list_second_target))
    p.x_range.start = 0  # start value of the x-axis

    p.xaxis.axis_label = "value of '" + column_target + "'"

    hover = HoverTool()  # initiate hover tool
    hover.tooltips = [("Value","@{" + column_target + "}{%.2f}" ),("Ranking","@index°")]
    hover.formatters={'@{'+column_target+'}' : 'printf'}

    hover.mode = 'hline' # set the mode of the hover tool
    p.add_tools(hover)   # add the hover tooltip to the plot

    return p # show in notebook

def eda_geoplot_country_rank_plot(the_df, primary_column, target_column, first_title, second_title, int_top = 10,
                                  location_column = 'Team/NOC', ):
    """
    Execute and show all together:
    @ primary_columns must to be a float to join to make a GeoSource
    generate_GeoJSONSource_to_districts()
    eda_seoul_districts_geo_plot()
    eda_bokeh_horiz_bar_ranked()
    """
    the_df = the_df.rename({primary_column: target_column}, axis = 1)

    geo_source = generate_GeoJSONSource(the_df)

    geo = eda_us_states_geo_plot(geo_source, the_df, first_title,
                                       target_column, location_column, palette = inferno(32))

    # rank 8 bottom and Up
    rank = eda_bokeh_horiz_bar_ranked(the_df, target_column, second_title,
                                      int_top = int_top, second_target = 'Team/NOC')

    show( row( geo, rank ))

## Import DataSet <a id='index03'></a> <a href="#top" style="background-color: #20beff; border: none; color: white;  padding: 15px 30px;  font-family: sans-serif;  border-radius: 12px;  text-align: center;  text-decoration: none;  display: inline-block;  font-size: 16px;  margin: 4px 2px;  cursor: pointer;" >Go to TOC</a>

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
medals_file = '/kaggle/input/olympic-games-2021-medals/Tokyo 2021 dataset.csv'
world_file = '/kaggle/input/world-map-eckert3/world-eckert3.geo.json'

In [ ]:
# Import Tokyo Medals file and Simple Exploratory Data Analysis
file_path = medals_file

df = pd.read_csv(file_path)
print("DataSet of Tokyo 2021 Medals = {:,d} rows and {} columns".format(df.shape[0], df.shape[1]))

print("\nAll Columns:\n=>", df.columns.tolist())

quantitative = [f for f in df.columns if df.dtypes[f] != 'object']
qualitative = [f for f in df.columns if df.dtypes[f] == 'object']

print("\nStrings Variables:\n=>", qualitative,
      "\n\nNumerics Variables:\n=>", quantitative)

df.head()

In [ ]:
df.head()

## Import GeoJson Of World <a id='index04'></a> <a href="#top" style="background-color: #20beff; border: none; color: white;  padding: 15px 30px;  font-family: sans-serif;  border-radius: 12px;  text-align: center;  text-decoration: none;  display: inline-block;  font-size: 16px;  margin: 4px 2px;  cursor: pointer;" >Go to TOC</a>

In [ ]:
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.models.tools import HoverTool
from bokeh.models import GeoJSONDataSource
from bokeh.layouts import row
output_notebook()

import geopandas as gpd

# import geojson
geojson = gpd.read_file(world_file)

# delete useless columns
list_to_delete = ['id', 'hc-group', 'hc-middle-x', 'hc-middle-y', 'hc-key', 'hc-a2',
                  'labelrank', 'woe-id', 'labelrank', 'iso-a3', 'iso-a2', 'woe-id']
geojson = geojson.drop(list_to_delete, axis = 1) 

# # Replace in GEO
replace_dict = {'United States of America': 'USA', 'United Republic of Tanzania': 'Tanzania',
                'Republic of Serbia': 'Serbia', 'The Bahamas':'Bahamas', 'United Kingdom': 'UK', 
               'Republic of Congo': 'Congo', 'Czech Republic': 'Czechia', 'South Korea': 'S. Korea',
}
geojson = geojson.dropna().replace(replace_dict)

# show
geojson.head(3)

In [ ]:
# fix df_tokyo 'Team/NOC' by correct countries
dict_replace_tokyo_medals = {
    'Syrian Arab Republic': 'Syria',
    'Republic of Korea': 'South Korea',
    'Republic of Moldova': 'Moldova',
    'Czech Republic': 'Czechia',
    'Islamic Republic of Iran': 'Iran',
    "Côte d'Ivoire": 'Ivory Coast',
    'Great Britain': 'UK',
    'United States of America': 'USA',
    "People's Republic of China": 'China',
    'ROC': 'Russia',
}
df['Team/NOC'] = df['Team/NOC'].replace(dict_replace_tokyo_medals)

In [ ]:
# Get GeoJSON, merge with tokyo_medals and out-put Bookeh GeoJSON
def generate_GeoJSONSource(my_df):
    global geojson
    geo_source_merged = geojson.merge(df, left_on = 'name', right_on = 'Team/NOC')
    with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
#         print(geo_source_merged)
        print(len(geo_source_merged))
    return GeoJSONDataSource( geojson = geo_source_merged.to_json())

In [ ]:
# Check If exist a name in 'df['Team/NOC']' and don't has in geojson['name']

df_column = 'Team/NOC'
geojson_country_column = 'name'

geo_source_merged = geojson.merge(df, left_on = geojson_country_column, right_on = df_column, how='right') # Join
geo_source_merged[geojson_country_column] = geo_source_merged[geojson_country_column].fillna('mising')

# list of contries in geoJSON
geojson_countries = geojson[geojson_country_column].tolist()
geojson_countries.sort()

# list of contries in COVID-DATASET
covid_data_countries = df[df_column].tolist()
covid_data_countries.sort()

data_in_tokyo_not_in_geo = []
for country in covid_data_countries:
    if(country not in geojson_countries):
        data_in_tokyo_not_in_geo.append(country)
        
data_in_geo_not_in_tokyo = []
for country in geojson_countries:
    if(country not in covid_data_countries):
        data_in_geo_not_in_tokyo.append(country)
        
print('\nHas in TOKYO but NOT IN GEO (SHOW TOKYO):\n', data_in_tokyo_not_in_geo)
print('\nHas in GEO but NOT IN TOKYO (SHOW GEO):\n', data_in_geo_not_in_tokyo)

In [ ]:
# Remove Contries from Tokyo CSV
df = df.drop(df[df['Team/NOC'].isin(data_in_tokyo_not_in_geo)].index, axis=0) # remove_list: removes mismatched data
df.head()

## Df with all countries

In [ ]:
df_geo_all_countries = geojson.merge(df, left_on = 'name', right_on = 'Team/NOC', how = "left")

for col in ['Gold Medal', 'Silver Medal', 'Bronze Medal', 'Total']:
    df_geo_all_countries[col] = df_geo_all_countries[col].fillna(0.0)
df_geo_all_countries['Team/NOC'] = df_geo_all_countries['name']
    
df_geo_all_countries.head()


## EDA: Map of Count Medals on World <a id='index05'></a> <a href="#top" style="background-color: #20beff; border: none; color: white;  padding: 15px 30px;  font-family: sans-serif;  border-radius: 12px;  text-align: center;  text-decoration: none;  display: inline-block;  font-size: 16px;  margin: 4px 2px;  cursor: pointer;" >Go to TOC</a>

### Top Gold Medals

In [ ]:
primary_col = 'Gold Medal'

df_aux = df.groupby(['Team/NOC']).sum()[primary_col].reset_index()

print(df_aux)

eda_geoplot_country_rank_plot(df_aux, primary_col, primary_col,
                           "Total Gold Medal in World", "The Top 20 Countries in Gold Medal")

### Silver Medal

In [ ]:
primary_col = 'Silver Medal'

df_aux = df.groupby(['Team/NOC']).sum()[primary_col].reset_index()

eda_geoplot_country_rank_plot(df_aux, primary_col, primary_col,
                           "Total Silver Medal in World", "The Top 20 Countries in Silver Medal")

### Bronze Medal

In [ ]:
primary_col = 'Bronze Medal'

df_aux = df.groupby(['Team/NOC']).sum()[primary_col].reset_index()

eda_geoplot_country_rank_plot(df_aux, primary_col, primary_col,
                           "Total Bronze Medal in World", "The Top 20 Countries in Bronze Medal")

### Count Total of Medals

In [ ]:
primary_col = 'Total'

df_aux = df.groupby(['Team/NOC']).sum()[primary_col].reset_index()

eda_geoplot_country_rank_plot(df_aux, primary_col, primary_col,
                           "Total of Medals on World", "The Top 20 Countries in Total of Medals")

### Web Scraping of Population <a id='index06'></a> <a href="#top" style="background-color: #20beff; border: none; color: white;  padding: 15px 30px;  font-family: sans-serif;  border-radius: 12px;  text-align: center;  text-decoration: none;  display: inline-block;  font-size: 16px;  margin: 4px 2px;  cursor: pointer;" >Go to TOC</a>

In [ ]:
import requests
!pip install bs4
from bs4 import BeautifulSoup

In [ ]:
website = 'https://www.worldometers.info/coronavirus/#countries' # url for the site 
website_url = requests.get(website).text
soup = BeautifulSoup(website_url,'html.parser')

In [ ]:
my_table = soup.find('tbody')

table_data = []
for row in my_table.findAll('tr'):
    row_data = []
    for cell in row.findAll('td'):
        row_data.append(cell.text)
    if(len(row_data) > 0):
        data_item = {"Country": row_data[1],
                     "TotalCases": row_data[2],
                     "NewCases": row_data[3],
                     "TotalDeaths": row_data[4],
                     "NewDeaths": row_data[5],
                     "TotalRecovered": row_data[6],
                     "ActiveCases": row_data[8],
                     "SeriousCritical": row_data[9],
                     "TotCases/1Mpop": row_data[10],
                     "Deaths/1Mpop": row_data[11],
                     "TotalTests": row_data[12],
                     "Tests/1Mpop": row_data[13],
                     "Population": row_data[14],
        }
        table_data.append(data_item)

dfr = pd.DataFrame(table_data)
dfr = dfr[8:] 
dfr.head()

In [ ]:
# columns = dfr.columns.tolist()

def remove_punctuation(x):
    y = x.replace(',','').replace('+','')
    return y

def a_function(x):
    return x.replace('\n','')

for c in dfr.columns.tolist():
    dfr[c] = dfr[c].apply(remove_punctuation).replace(r'^\s*$', np.nan, regex=True)

dfr.head()

In [ ]:
for c in dfr.columns.tolist()[1:]: # excet Countries
    dfr[c] = dfr[c].fillna(0).replace('N/A','')
    dfr[c] = dfr[c].fillna({'':0}).replace('',0)
    dfr[c] = dfr[c].astype(float).astype(int)
    
dfr = dfr.reset_index(drop=True)

In [ ]:
df_population = dfr[['Country', 'Population']]
print('count', len(df_population))
df_population.head()

## EDA: Medals by Population <a id='index07'></a> <a href="#top" style="background-color: #20beff; border: none; color: white;  padding: 15px 30px;  font-family: sans-serif;  border-radius: 12px;  text-align: center;  text-decoration: none;  display: inline-block;  font-size: 16px;  margin: 4px 2px;  cursor: pointer;" >Go to TOC</a>

### Insert Population in DataFrame

In [ ]:
df.head()

In [ ]:
df_with_population = df.merge(df_population, left_on = 'Team/NOC', right_on = 'Country')
df_with_population

In [ ]:
# Get GeoJSON, merge with tokyo_medals and out-put Bookeh GeoJSON
def new_generate_GeoJSONSource(my_df):
    global geojson
    xeo_source_merged = geojson.merge(my_df, left_on = 'name', right_on = 'Team/NOC')
    with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
#         print(xeo_source_merged)
        print(len(xeo_source_merged))
    return GeoJSONDataSource( geojson = xeo_source_merged.to_json()), xeo_source_merged

In [ ]:
def new_eda_geoplot_country_rank_plot(o_df, primary_column, target_column, first_title, second_title, int_top = 10,
                                  location_column = 'Team/NOC', ):
    """
    Execute and show all together:
    @ primary_columns must to be a float to join to make a GeoSource
    generate_GeoJSONSource_to_districts()
    eda_seoul_districts_geo_plot()
    eda_bokeh_horiz_bar_ranked()
    """
    o_df = o_df.rename({primary_column: target_column}, axis = 1)
    
    print(o_df.columns.tolist())
    
#     print(the_df)
    geo_source, the_new_df = new_generate_GeoJSONSource(o_df)
    
    print(the_new_df.columns.tolist())

#     geo_source = generate_GeoJSONSource(the_df)
    # cuidado, pois o 'geo_source' vem de um merge que pode retirar dados,se for diferente, ento o mapa GEO fica todo branco

    geo = eda_us_states_geo_plot(geo_source, the_new_df, first_title,
                                       target_column, location_column, palette = inferno(32))

    # rank 8 bottom and Up
    rank = eda_bokeh_horiz_bar_ranked(the_new_df, target_column, second_title,
                                      int_top = int_top, second_target = 'Team/NOC')
    
    print(rank)
    show(rank)
#     show( row(geo, rank ))

In [ ]:
primary_col = 'totalpop'

df_with_population['totalpop'] = (df_with_population['Total'] / df_with_population['Population'] ) *100000000

df_aux3 = df_with_population.groupby(['Team/NOC']).sum()[primary_col].reset_index()

print(df_aux3)

new_eda_geoplot_country_rank_plot(df_aux3, primary_col, primary_col,
                           "Total Gold Medal in World", "The Top 20 Countries in Gold Medal")

In [ ]:
geojson